In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt

### Dataset: Goodread 10k
Because Collaborative Filtering required a huge dataset of user-item interaction, so we going to use a dataset called "Goodread 10k" which contain 10000 unique books, 53000 unique users and 1 million interaction logs between user-item

https://www.kaggle.com/jealousleopard/goodreadsbooks

In [2]:
rating = pd.read_csv("./data/goodread/ratings.csv")
books = pd.read_csv("./data/goodread/books.csv")
books = books[['id','authors','title','average_rating','ratings_count']].rename(columns={'id':'book_id'})

In [3]:
books.sample(20)

,book_id,authors,title,average_rating,ratings_count
3134,3135,Bill Bryson,"One Summer: America, 1927",4.06,27237
6898,6899,J.D. Horn,"The Line (Witching Savannah, #1)",3.78,7834
9674,9675,"Brian Herbert, Kevin J. Anderson",House Corrino (Prelude to Dune #3),3.64,10682
1380,1381,Jasper Fforde,"The Eyre Affair (Thursday Next, #1)",3.92,85683
6572,6573,Leil Lowndes,How to Talk to Anyone: 92 Little Tricks for Bi...,3.80,12206
8368,8369,"Melissa Anelli, J.K. Rowling","Harry, a History: The True Story of a Boy Wiza...",4.09,12308
7579,7580,"Tom Sniegoski, Jeff Smith, Thomas E. Sniegoski...","Bone: Quest for the Spark, Vol. 1",4.33,11416
5082,5083,Daniel Silva,"The Messenger (Gabriel Allon, #6)",4.18,16974
320,321,Wilson Rawls,Where the Red Fern Grows,4.04,268548
2131,2132,"Eliyahu M. Goldratt, Jeff Cox",The Goal: A Process of Ongoing Improvement,4.00,34837


In [4]:
rating.sample(20)

,book_id,user_id,rating
709845,7142,43906,3
83298,834,2066,4
692918,6969,26606,2
899874,9118,30224,3
589119,5915,8352,5
980291,9985,25821,4
935625,9502,35621,5
920954,9343,13786,5
169798,1699,6758,3
324115,3245,11472,5


In [5]:
num_users = len(rating['user_id'].unique())
num_books = len(rating['book_id'].unique())
print("Number of user:",num_users)
print("Number of books:",num_books)

Number of user: 53424
Number of books: 10000


![title](./assets/utility-matrix.png)

In [6]:
cor = rating[['user_id','book_id']].values
# -1 because our dataset count index from 1
# but python index count from 0
cor -= 1
val = rating['rating'].values

In [7]:
rating_matrix = coo_matrix((val, (cor[:,0],cor[:,1]))).toarray()

In [8]:
rating_matrix.shape

(53424, 10000)

## Build a Recommendation base on similar users
![title](./assets/alsoview.png)

In [9]:
def get_book_read_by_user(user_id,with_book_info=False):
    user_books = rating[rating['user_id']==user_id].reset_index(drop=True)
    read_books_id = user_books['book_id'].values
    if not with_book_info:
        return read_books_id
    read_books_info = books[books['book_id'].isin(read_books_id)].reset_index(drop=True)
    read_books_info = pd.merge(user_books,read_books_info,on='book_id')
    return read_books_info

In [10]:
def get_similar_user(user_id,k=10):
    # -1 rating table
    _id = user_id - 1
    idx = np.where(rating_matrix[_id]!=0)[0]
    sliced_rating_matrix = rating_matrix[:,idx]
    target = sliced_rating_matrix[_id]
    
    dist = [np.squeeze(euclidean_distances(target.reshape(1,-1),u.reshape(1,-1))) for u in sliced_rating_matrix]
    
    similar_user = np.argsort(dist)
    similar_user += 1
    return similar_user[:k]

In [11]:
def similar_books(user_1,user_2):
    u1 = get_book_read_by_user(user_1)
    u2 = get_book_read_by_user(user_2)
    return np.intersect1d(u1,u2)

In [12]:
target_user = 412
get_book_read_by_user(412,True)

,book_id,user_id,rating,authors,title,average_rating,ratings_count
0,3930,412,4,"Aristotle, J.A.K. Thomson, Jonathan Barnes, Hu...",The Nicomachean Ethics,3.91,19380
1,4640,412,5,Raymond Carver,Cathedral,4.30,18378
2,4908,412,2,Neil Strauss,The Game: Penetrating the Secret Society of Pi...,3.73,16698
3,5122,412,4,Hunter S. Thompson,Fear and Loathing on the Campaign Trail '72,4.10,15306
4,7379,412,3,Steve Martin,The Pleasure of My Company,3.78,12378
5,7771,412,4,Lorrie Moore,Birds of America,4.12,10768
6,8676,412,5,Richard Price,Lush Life,3.69,9478
7,8703,412,4,Adrian Nicole LeBlanc,"Random Family: Love, Drugs, Trouble, and Comin...",4.23,9053
8,8878,412,3,"Gabriel García Márquez, J.S. Bernstein",No One Writes to the Colonel and Other Stories,3.86,10480
9,9533,412,4,Sherman Alexie,Reservation Blues,3.98,9919


In [13]:
get_similar_user(412,k=10)

array([  412, 39032,  3222,  2645,   491,  1861,  8284, 28563, 44496,
       24040])

In [14]:
similar_books(412,39032)

array([4640, 7771, 9759])

In [15]:
get_book_read_by_user(3222,True)

,book_id,user_id,rating,authors,title,average_rating,ratings_count
0,187,3222,3,Scott Westerfeld,"Uglies (Uglies, #1)",3.86,449073
1,192,3222,4,Patrick Rothfuss,The Name of the Wind (The Kingkiller Chronicle...,4.55,400101
2,379,3222,3,Deborah Harkness,"A Discovery of Witches (All Souls Trilogy, #1)",3.99,226622
3,380,3222,4,"Haruki Murakami, Jay Rubin",Norwegian Wood,4.02,183988
4,422,3222,5,J.K. Rowling,"Harry Potter Boxset (Harry Potter, #1-7)",4.74,190050
5,430,3222,4,"Haruki Murakami, Philip Gabriel",Kafka on the Shore,4.13,167593
6,492,3222,3,Orson Scott Card,"Speaker for the Dead (Ender's Saga, #2)",4.04,164287
7,519,3222,3,Scott Westerfeld,"Pretties (Uglies, #2)",3.85,193405
8,520,3222,4,"Haruki Murakami, Jay Rubin",The Wind-Up Bird Chronicle,4.17,133408
9,572,3222,4,Margaret Atwood,"Oryx and Crake (MaddAddam, #1)",4.00,151500


## Matrix Factorization

![title](./assets/matrix-factorization.png)


### Why Matrix Factorization

#### 1. Smaller Size

   For example, if we have 53424 users, 10000 items (books) => our rating matrix will have shape of $(U,I) = (53424,10000)$. If each cell is a 'float64' number:

  $$total\_memory = 53424\cdot10000\cdot8$$
  $$total\_memory = 4273920000(b)$$
  $$total\_memory \approx 4,274(gb)$$
  
   If we can factorize this $(U,I)$ matrix to $(U,k)\cdot(I,k)^T$ with $k = 32$ (for example), then we will greatly reduce our space complexity:
   
   $$total\_memory = 53424\cdot32\cdot8 + 10000\cdot32\cdot8$$
   $$total\_memory = 16236544(b)$$
   $$total\_memory \approx 16(mb)$$


#### 2. Predict Items Rating (Matrix completion)
   Make inference time much faster. Instead of calculate similar users to a given target user, then sample a list of recommendation from these similar users, the product of 2 latent matrices will give us predicted rating score of each user to each item.
   
   
#### 3. "Meaningful" latent vectors

users' preferences, items' features
   
   From an application point of view, matrix factorization can be used to discover latent features underlying the interactions between two different kinds of entities. (Of course, you can consider more than two kinds of entities and you will be dealing with tensor factorization, which would be more complicated.) And one obvious application is to predict ratings in collaborative filtering
   
   More info: http://nicolas-hug.com/blog/matrix_facto_1

## Singular Value Decomposition (SVD)

![title](./assets/svd.png)

https://en.wikipedia.org/wiki/Singular_value_decomposition

https://research.fb.com/blog/2014/09/fast-randomized-svd/

Note: Definition of SVD in Recommendation System often doesn't equal its definition in Mathematic.  

$A \approx U\cdot\Sigma\cdot{V^T}$

Where:

   - A is our utility matrix that has shape $(m,n)$

   - U is an matrix that has shape $(m,k)$

   - V is an matrix that has shape $(n,k)$
   
   - $\Sigma$ have shape $(k,k)$
   
## FunkSVD

https://sifter.org/~simon/journal/20061211.html

https://medium.com/datadriveninvestor/how-funk-singular-value-decomposition-algorithm-work-in-recommendation-engines-36f2fbf62cac

Invented by Simson Funk, who shared his this techinque to the public after the Netflix Prize 2006. 

Quite similar to SVD, FunkSVD try to factorized a huge utility matrix $A(m,n)$ to 2 (instead of 3 like SVD) smaller matrix $U(m,k)$ and $V(n,k)$ where $k < m | n$

$A \approx U\cdot{V^T}$

However, What make FunkSVD special is its ability to ignore unobserved data. SVD doesn't work very well with extreme sparse utility matrix.

In [16]:
observed_data = len(rating)
num_cell = np.prod(rating_matrix.shape)
sparsity = (1 - observed_data/num_cell)
print("Number of observed interactions:",observed_data)
print("Number of cell in the rating matrix:",num_cell)
print("Sparsity: {:.02f}%".format(sparsity*100))

Number of observed interactions: 981756
Number of cell in the rating matrix: 534240000
Sparsity: 99.82%


In [17]:
del rating_matrix

In [18]:
import tensorflow as tf
import sys

In [19]:
tf.reset_default_graph()

### Create Sparsed Rating Matrix
We don't want to storge 4gb matrix which contain mostly 0 into our memory, this is just a waste of computing resource. So we will represent our rating matrix as a 'sparsed' format

Sparsed matrix is just a collection of 3 smaller matrices
- 1st matrix represent all the indices (cell cordinate) where there is a value, ex: [[0,0], [32,1], [2,3214]]
- 2nd matrix represent all the value in the indices matrix above, ex: [5,3,4]
- 3rd matrix represent the over all shape of the true rating matrix, ex: [53424,10000] 

In [20]:
def create_sparse_matrix(num_user,num_item):
    dense_shape = (num_user,num_item)
    resp =  tf.SparseTensor(cor,val,dense_shape)
    return resp

In [21]:
sparse_rating_matrix = create_sparse_matrix(num_users,num_books)

In [22]:
k_latent = 32

In [23]:
U = tf.get_variable("User",shape=[num_users,k_latent],dtype=tf.float32,initializer=tf.initializers.glorot_uniform())
V = tf.get_variable("Item",shape=[num_books,k_latent],dtype=tf.float32,initializer=tf.initializers.glorot_uniform())

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
# Measure how similar our prediction to the true rating score
def sparsed_rmse(pred,true):
    return tf.sqrt(tf.losses.mean_squared_error(true,pred))

### Computation Graph
1. Compute the dot product of our random $U$ and $V$
2. Measure how good our prediction are to the true rating matrix with our score function **sparsed_rmse**
3. Update $U$ and $V$ via **Gradient Descent** to minimize loss
4. Repeat until we satisfied with our result

In [25]:
# This will return a matrix with shape (num_users, num_books) - (53424,1000)
predicted_rating_matrix = tf.matmul(U,V,transpose_b=True)

# However we only care about all the indices where we have observed data
predicted_rating_matrix_sparse = tf.gather_nd(predicted_rating_matrix, cor)

# Calculate the loss
# the bigger the loss, the less similar our predicted rating matrix are to the true rating matrix
loss = sparsed_rmse(predicted_rating_matrix_sparse,sparse_rating_matrix.values)

# Create our optimizer and minimize our loss
op = tf.train.AdamOptimizer(learning_rate=0.1)
train = op.minimize(loss)

Instructions for updating:
Use tf.cast instead.


In [26]:
# initialize all our variables and run out computation graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [27]:
EPOCHS = 300
for i in range(1,EPOCHS+1):
    loss_val, _ = sess.run([loss,train])
    sys.stdout.write(f"\rEpoch{i}: loss = {loss_val}")
    if (i % 20 == 0):
        print(f"\rEpoch {i}: loss = {loss_val}")

Epoch20: loss = 0.8205154538154602
Epoch40: loss = 0.41893976926803596
Epoch60: loss = 0.26165738701820374
Epoch80: loss = 0.19391773641109467
Epoch100: loss = 0.15845872461795807
Epoch120: loss = 0.15550963580608368
Epoch140: loss = 0.14832806587219238
Epoch160: loss = 0.12932938337326056
Epoch180: loss = 0.12611706554889682
Epoch200: loss = 0.12435486912727356
Epoch220: loss = 0.11753682792186737
Epoch240: loss = 0.11702726781368256
Epoch260: loss = 0.11372736841440201
Epoch280: loss = 0.11083818227052689
Epoch300: loss = 0.11015793681144714


In [35]:
U = np.load("User_latent.npy")
V = np.load("Item_latent.npy")

In [57]:
target_user = 42135
target_user_latent = U[target_user-1].reshape(1,-1)
target_user_latent.shape

(1, 32)

In [58]:
predicted_rating = np.dot(target_user_latent,V.T)[0]

In [59]:
recommended_books = np.argsort(predicted_rating)[::-1]
recommended_books_rating = np.sort(predicted_rating)[::-1]

recommended_books = recommended_books[:20]
recommended_books_rating = recommended_books_rating[:20]

recommended_books+=1

In [60]:
get_book_read_by_user(target_user,True)

,book_id,user_id,rating,authors,title,average_rating,ratings_count
0,5502,42135,4,"Julietta Suzuki, Tomo Kimura","Kamisama Kiss, Vol. 1",4.38,20569
1,5653,42135,3,"Akira Amano, JN Productions, Frances E. Wall","Reborn! Vol. 01: Reborn Arrives! (Reborn!, #1)",4.22,15102
2,6452,42135,1,"Tachibana Higuchi, 樋口 橘","Gakuen Alice, Vol. 01 (Gakuen Alice, #1)",4.23,15373
3,7076,42135,2,"Jinsei Kataoka, Kazuma Kondou","Deadman Wonderland, Volume 1 (Deadman Wonderla...",4.20,12932
4,7688,42135,2,Hidekaz Himaruya,"Hetalia: Axis Powers, Vol. 1 (Hetalia: Axis Po...",4.35,11985
5,9695,42135,3,Kiyo Fujiwara,"Wild Ones, Vol. 1 (Wild Ones, #1)",4.03,10738
6,9730,42135,5,Bisco Hatori,"Millennium Snow, Vol. 1",4.04,11293
7,9781,42135,3,Hiro Fujiwara,Maid-sama! Vol. 02 (Maid-sama! #2),4.53,9843


In [61]:
temp = books[books['book_id'].isin(recommended_books)]
temp.insert(5,'user_predicted_rating',recommended_books_rating)
temp

,book_id,authors,title,average_rating,ratings_count,user_predicted_rating
48,49,Stephenie Meyer,"New Moon (Twilight, #2)",3.52,1149630,8.968071
55,56,Stephenie Meyer,"Breaking Dawn (Twilight, #4)",3.70,1070245,8.006418
260,261,Frances Mayes,Under the Tuscan Sun,3.72,279264,7.713980
365,366,David McCullough,John Adams,4.05,215780,7.712398
1676,1677,Michael Crichton,Next,3.48,53487,7.710805
1832,1833,Nancy E. Turner,These Is My Words: The Diary of Sarah Agnes Pr...,4.35,44669,7.645083
2321,2322,Lisa See,Dreams of Joy (Shanghai Girls #2),4.06,43863,7.421241
2542,2543,David Mitchell,The Thousand Autumns of Jacob de Zoet,4.03,36021,7.256475
3295,3296,Joshua Ferris,Then We Came to the End,3.45,23614,7.209743
3637,3638,Marilynne Robinson,Housekeeping,3.82,28416,7.193165


In [ ]:
from sklearn.utils.extmath import randomized_svd

In [ ]:
#time it
U, Sigma, V_T = randomized_svd(rating_matrix,n_components=128)
Sigma = np.diag(Sigma)

In [ ]:
print(U.shape)
print(Sigma.shape)
print(V_T.shape)

In [ ]:
# 23122, 42135 (manga)
# 2123 (non-fiction)
# 5231 (author-bias)
# 42152 (cold start)
target_user = 2123
target_user_latent = U[target_user-1].reshape(1,-1)
target_user_latent.shape

In [ ]:
predicted_rating = np.dot(np.dot(target_user_latent,Sigma),V_T)[0]

In [ ]:
recommended_books = np.argsort(predicted_rating)[::-1]
recommended_books_rating = np.sort(predicted_rating)[::-1]

recommended_books = recommended_books[:20]
recommended_books_rating = recommended_books_rating[:20]

recommended_books+=1

In [ ]:
get_users_read_book(target_user,True)

In [ ]:
temp = books[books['book_id'].isin(recommended_books)]
temp.insert(5,'user_predicted_rating',recommended_books_rating)
temp

# Conclusion

### Collaborative Filtering / Matrix Factorization

#### Pros:

- Smaller memory footprint
   
- Efficient query time (after the factorization step is complete)
   
- "Meaningful" latent space, helpful when compute similar users, similar items
   
- Doesn't require any meta data (item's title, description, tags,... ). Derived users' and items' features directly from the interaction matrix alone.

   
#### Cons:
    
- Popularity bias: Super popular items have more iteraction in the utility matrix => our model will start recommend some super popular items to everyone just because most people have interact with these items.

- Cold-start problems: As oppose to popular items, items that have small number of interactions will never get recommened to anyone. The same thing is true to user, if an user that have small number of interactions, the recommended items will not accurate.
